In [1]:
# Dependencies
import tweepy
import json
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


# from config import get_tweepy_api

# # Setup Tweepy API Authentication
# api = get_tweepy_api()

In [4]:
def filter_human(user):
    min_tweets = 5
    max_tweets = 10000
    max_followers = 2500
    max_following = 2500
    lang = "en"
    
    return user["followers_count"] < max_followers and\
        user["statuses_count"] > min_tweets and\
        user["statuses_count"] < max_tweets and\
        user["friends_count"] < max_following and\
        user["lang"] == lang


In [ ]:
# Target Search Term
target_terms = ("@BBCWorld", "@CBSNews", "@CNN",
                "@FoxNews", "@nytimes")

# List to hold sentiment
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (filter_human(tweet["user"])):
                
                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                results_list.append({
                    "Date": tweet["created_at"],
                "Compound": results["compound"],
                "Positive": results["pos"],
                "Negative": results["neg"],
                "Neutral": results["neu"]
                })

#                 # Run Vader Analysis on each tweet
#                 results = analyzer.polarity_scores(tweet["text"])
#                 compound = results["compound"]
#                 pos = results["pos"]
#                 neu = results["neu"]
#                 neg = results["neg"]

#                 # Add each value to the appropriate list
#                 compound_list.append(compound)
#                 positive_list.append(pos)
#                 negative_list.append(neg)
#                 neutral_list.append(neu)
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1

#     # Store the Average Sentiments
#     sentiment = {
#         "User": target,
#         "Compound": compound_list,
#         "Positive": positive_list,
#         "Neutral": mean(negative_list,
#         "Negative": neutral_list,
#     }

#     # Print the Sentiments
#     print(sentiment)
#     print()
    
#     # Append airline results to 'results_list'
#     results_list.append(sentiment)


In [ ]:
# Create a DataFrame using results_list and display
news_df = pd.DataFrame(results_list).set_index("User")
news_df.head()